# Analyse des résultats de la  prédiction

In [1]:
#récupération de l'ap de bigml

from bigml.api import BigML
from pandas import read_csv
from sklearn.metrics import confusion_matrix
import pandas as pan
import matplotlib.pyplot as plt
import random

In [2]:
#récupération des identifiants du projet de bigml

%env BIGML_USERNAME=damienthiberge
%env BIGML_API_KEY=d92377e535638e44239dc49238540477263e1bff

env: BIGML_USERNAME=damienthiberge
env: BIGML_API_KEY=d92377e535638e44239dc49238540477263e1bff


In [3]:
#charge de l'api bigml pour pouvoir faire appel à ces fonctions

api = BigML()

### récupération du fichier train full

In [6]:
#récupération du fichier csv bigml à traiter

train_full = api.create_source('./train_full.csv')

In [7]:
#test des sources

api.ok(train_full)

True

In [8]:
#création dataset depuis le fichier csv importer précédement

train_full_dataset = api.create_dataset(train_full)

### split du fichier train full afin d'entrainer la machine

In [9]:
#split du train full

train_dataset = api.create_dataset(
    train_full_dataset, {"name": "Train (80%)",
                     "sample_rate": 0.8, "seed": "Full train"})
validation_dataset = api.create_dataset(
    train_full_dataset, {"name": "Train (20%)",
                     "sample_rate": 0.8, "seed": "Full train",
                     "out_of_bag": True})

### création du model puis batch prédiction

In [10]:
#creation d'un ensemble à partir de train 

ensemble = api.create_ensemble([train_dataset],{"objective_field":"SeriousDlqin2yrs"})

In [11]:
#batch prédiction

batch_prediction = api.create_batch_prediction(ensemble, validation_dataset, {"all_fields":True, "probabilities":True, 'prediction_name':'Prediction'})


In [12]:
#test prediction

api.ok(batch_prediction)

True

### récupération csv de la prédiction

In [13]:
#telechargement de la prédiction

api.download_batch_prediction(batch_prediction,
    filename='../../handson-ml/predictions_error.csv')

'../../handson-ml/predictions_error.csv'

### mise en forme de la prédiction

In [14]:
error = read_csv('../../handson-ml/predictions_error.csv', index_col=0)
error.to_csv('../../handson-ml/predictions_error.csv', index=False)

# renomme la colonne 1 probability en prédiction
myList = list(error.columns)
myList[1] = 'Prediction'
error.columns = myList

# création de la colonne error
error.insert( 2, 'Error', '')

In [ ]:
# fonction de remplissage de la colonne error
def errorPred(row):
    if row['SeriousDlqin2yrs'] == 0 and row['Prediction'] == 0:
        row['Error'] = 'TN'
    if row['SeriousDlqin2yrs'] == 0 and row['Prediction'] == 1:
        row['Error'] = 'FP'
    if row['SeriousDlqin2yrs'] == 1 and row['Prediction'] == 0:
        row['Error'] = 'FN'
    if row['SeriousDlqin2yrs'] == 1 and row['Prediction'] == 1:
        row['Error'] = 'TP'
    return row['Error']


In [ ]:
# remplissage de la colonne error
error['Error'] = error.apply(errorPred,axis=1)

In [ ]:
error

### calcul de la matrice de confusion, accurancy et gain

In [ ]:
# affichage de la matrice de confusion
resultat = error['Error'].value_counts()
resultat

In [ ]:
# calcul de l'accurancy
accuracy = (resultat[0] + resultat[2])/(resultat[0] + resultat[1] + resultat[2] + resultat[3])
accuracy

In [ ]:
# calcul de la matrice de gain
matrice_gain = resultat[0]*500 - resultat[1]*500 + resultat[2]*0 - resultat[3]*2500
matrice_gain

### calcul gain en fonction d'une valeur seuil

# fonction de remplissage de la colonne error en prenant en compte une valeur seuil et retour du gain
def gain_seuil(seuil):
    def seuil_pred(row, seuil):
        if row['1 probability'] > seuil and row['SeriousDlqin2yrs'] == 0:
            row['Error'] = 'FP'
        if row['1 probability'] > seuil and row['SeriousDlqin2yrs'] == 1:
            row['Error'] = 'TP'
        if row['1 probability'] < seuil and row['SeriousDlqin2yrs'] == 0:
            row['Error'] = 'TN'
        if row['1 probability'] < seuil and row['SeriousDlqin2yrs'] == 1:
            row['Error'] = 'FN'
        return row

    for df in [error]:
        df['Error'] = df[['Error', '1 probability', 'SeriousDlqin2yrs']].apply(seuil_pred, axis=1, seuil = seuil)

    count = error['Error'].value_counts()
    gain = 1
    if "TP" not in  error['Error']:
        if "FP" not in  error['Error']:
            gain = count[0]*500 - count[1]*2500
        else:
            gain = count[0]*500 - count[1]*2500 - count[2]*500
    else:
        gain = count[0]*500 - count[1]*2500 - count[3]*500
        
    return gain

# affichage de la courbe de gzin en fonction de la valeur seuil
table = [gain_seuil(n/100) for n in range(1,100)]
seuil = [(n/100) for n in range(1,100)]
plt.plot(seuil, table)
plt.show()

# calcul de la valeur max du gain et obtention du seuil optimal
max_val = max(table)
max_seuil = table.index(max_val)/100

print(f"le gain maximal est de {max_val} et il est obtenu en {max_seuil}")

In [ ]:
# compter les valeurs dans la colonne error
from collections import Counter

c = Counter(error['SeriousDlqin2yrs'])
c_pred = Counter(error['Error'])
c_pred

### calcul de l'AUC (area under the courbe)

In [ ]:
pos = error.loc[error['SeriousDlqin2yrs'] == 1] # Tous les Positif  
neg = error.loc[error['SeriousDlqin2yrs'] == 0] # Tous les negatifs  
x = 0 
y = 0 # pour chaque 1 proba ( P ) dans toutes les 1 proba (P)
for threshold_pos in pos['1 probability']: # pour chaque 1 Proba ( N ) dans toutes les 1 proba ( N )     
    for threshold_neg in neg['1 probability']:         
        if threshold_pos > threshold_neg:             
            x += 1          
        y += 1 
print(x/y)